https://datahack.analyticsvidhya.com/contest/practice-problem-big-mart-sales-iii/

In [1]:
import numpy as np
import pandas as pd
train = pd.read_csv('./dataset/big_mart_sales/train.csv')
test = pd.read_csv('./dataset/big_mart_sales/test.csv')

In [2]:
train.shape

(8523, 12)

In [3]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
Item_Identifier              8523 non-null object
Item_Weight                  7060 non-null float64
Item_Fat_Content             8523 non-null object
Item_Visibility              8523 non-null float64
Item_Type                    8523 non-null object
Item_MRP                     8523 non-null float64
Outlet_Identifier            8523 non-null object
Outlet_Establishment_Year    8523 non-null int64
Outlet_Size                  6113 non-null object
Outlet_Location_Type         8523 non-null object
Outlet_Type                  8523 non-null object
Item_Outlet_Sales            8523 non-null float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.1+ KB


In [5]:
train['source'] = 'train'
test['source'] = 'test'
data = pd.concat([train,test],ignore_index=True)
print train.shape, test.shape, data.shape

(8523, 13) (5681, 12) (14204, 13)


In [6]:
data.apply(lambda x: sum(x.isnull()))

Item_Fat_Content                0
Item_Identifier                 0
Item_MRP                        0
Item_Outlet_Sales            5681
Item_Type                       0
Item_Visibility                 0
Item_Weight                  2439
Outlet_Establishment_Year       0
Outlet_Identifier               0
Outlet_Location_Type            0
Outlet_Size                  4016
Outlet_Type                     0
source                          0
dtype: int64

In [7]:
data.describe()

,Item_MRP,Item_Outlet_Sales,Item_Visibility,Item_Weight,Outlet_Establishment_Year
count,14204.000000,8523.000000,14204.000000,11765.000000,14204.000000
mean,141.004977,2181.288914,0.065953,12.792854,1997.830681
std,62.086938,1706.499616,0.051459,4.652502,8.371664
min,31.290000,33.290000,0.000000,4.555000,1985.000000
25%,94.012000,834.247400,0.027036,8.710000,1987.000000
50%,142.247000,1794.331000,0.054021,12.600000,1999.000000
75%,185.855600,3101.296400,0.094037,16.750000,2004.000000
max,266.888400,13086.964800,0.328391,21.350000,2009.000000


In [8]:
data.apply(lambda x : len(x.unique()))

Item_Fat_Content                 5
Item_Identifier               1559
Item_MRP                      8052
Item_Outlet_Sales             3494
Item_Type                       16
Item_Visibility              13006
Item_Weight                    416
Outlet_Establishment_Year        9
Outlet_Identifier               10
Outlet_Location_Type             3
Outlet_Size                      4
Outlet_Type                      4
source                           2
dtype: int64

In [9]:
categorical_variables=data.dtypes[data.dtypes=='object'].index
categorical_variables

Index([u'Item_Fat_Content', u'Item_Identifier', u'Item_Type',
       u'Outlet_Identifier', u'Outlet_Location_Type', u'Outlet_Size',
       u'Outlet_Type', u'source'],
      dtype='object')

In [10]:
categorical_variables=data.dtypes[data.dtypes=='object'].index
categorical_variables = [x for x in categorical_variables if x not in ['Item_Identifier','Outlet_Identifier','source'] ]
print categorical_variables
for var in categorical_variables:
    print '\nCategorical Variable - %s : '%var
    print data[var].value_counts(dropna=False)

['Item_Fat_Content', 'Item_Type', 'Outlet_Location_Type', 'Outlet_Size', 'Outlet_Type']

Categorical Variable - Item_Fat_Content : 
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Categorical Variable - Item_Type : 
Fruits and Vegetables    2013
Snack Foods              1989
Household                1548
Frozen Foods             1426
Dairy                    1136
Baking Goods             1086
Canned                   1084
Health and Hygiene        858
Meat                      736
Soft Drinks               726
Breads                    416
Hard Drinks               362
Others                    280
Starchy Foods             269
Breakfast                 186
Seafood                    89
Name: Item_Type, dtype: int64

Categorical Variable - Outlet_Location_Type : 
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Categorical Variable - Outlet_Size : 
Medium    4655
NaN       4016
S

In [11]:
print 'Categorical Variable - Item_Fat_Content :'
print data['Item_Fat_Content'].value_counts()

Categorical Variable - Item_Fat_Content :
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64


In [12]:
def getItemWeighByGroup(g):
    return g['Item_Weight'].mean()
fillItemWeight = lambda x: x.fillna(getItemWeighByGroup(x))
data['Item_Weight']=data.groupby('Item_Identifier').apply(fillItemWeight)['Item_Weight']

In [13]:
len(data[data['Item_Weight'].isnull()])

0

In [14]:
data.loc[:10,['Outlet_Size','Outlet_Type']]

,Outlet_Size,Outlet_Type
0,Medium,Supermarket Type1
1,Medium,Supermarket Type2
2,Medium,Supermarket Type1
3,NaN,Grocery Store
4,High,Supermarket Type1
5,Medium,Supermarket Type2
6,High,Supermarket Type1
7,Medium,Supermarket Type3
8,NaN,Supermarket Type1
9,NaN,Supermarket Type1


In [15]:
data.loc[data['Outlet_Type']=='Grocery Store','Outlet_Size'].value_counts()

Small    880
Name: Outlet_Size, dtype: int64

In [16]:
# mode
def getOutletSizeByGroup(g):
    return (g['Outlet_Size'].value_counts()).index[0]
fillOutletSize = lambda g: g.fillna(getOutletSizeByGroup(g))
data['Outlet_Size']=data.groupby('Outlet_Type').apply(fillOutletSize)['Outlet_Size']

In [17]:
data.loc[:10,['Outlet_Size','Outlet_Type']]

,Outlet_Size,Outlet_Type
0,Medium,Supermarket Type1
1,Medium,Supermarket Type2
2,Medium,Supermarket Type1
3,Small,Grocery Store
4,High,Supermarket Type1
5,Medium,Supermarket Type2
6,High,Supermarket Type1
7,Medium,Supermarket Type3
8,Small,Supermarket Type1
9,Small,Supermarket Type1


In [18]:
len(data[data['Outlet_Size'].isnull()])

0

In [19]:
grouped = data.groupby('Outlet_Type')
grouped['Item_Outlet_Sales'].agg(['mean','count'])

,mean,count
Outlet_Type,,
Grocery Store,339.828500,1083
Supermarket Type1,2316.181148,5577
Supermarket Type2,1995.498739,928
Supermarket Type3,3694.038558,935


In [20]:
# Item_Visibility - outlier - zero
visibility_zero = (data['Item_Visibility'] == 0)
visibility_means =data[visibility_zero!=1].groupby('Item_Identifier')['Item_Visibility'].mean()
# = lambda x: data[data['Item_Identifier']==x['Item_Identifier']][Item_Visibility]
getVisibility = lambda x: visibility_means[x['Item_Identifier']]
data.loc[visibility_zero,'Item_Visibility']=data[visibility_zero].apply(getVisibility,axis=1)

In [21]:
# create new feature Item Visibility Mean Ratio by Store
getVisibilityMeanRatio = lambda x: x['Item_Visibility']/visibility_means[x['Item_Identifier']]
#data['Item_Visibility_Mean_Ratio'] = data.groupby('Store_Identifier')['Item_Visibility'].transform(getVisibilityMeanRatio,axis=1)
data['Item_Visibility_Mean_Ratio']=data.apply(getVisibilityMeanRatio,axis=1)

In [22]:
#data.loc[data['Outlet_Identifier']=='OUT049',['Item_Visibility','Item_Visibility_Mean_Ratio']]
data['Item_Visibility_Mean_Ratio'].describe()

count    14204.000000
mean         1.000000
std          0.200592
min          0.803455
25%          0.901974
50%          0.925261
75%          1.000000
max          1.614811
Name: Item_Visibility_Mean_Ratio, dtype: float64

In [23]:
data.loc[:5,['Item_Visibility','Item_Visibility_Mean_Ratio']]

,Item_Visibility,Item_Visibility_Mean_Ratio
0,0.016047,0.931078
1,0.019278,0.933420
2,0.016760,0.864062
3,0.022930,1.000000
4,0.014670,1.000000
5,0.091294,1.000000


In [24]:
data['Item_Identifier'].str[:2].unique()

array(['FD', 'DR', 'NC'], dtype=object)

In [25]:
data['Item_Type_Combined'] = data['Item_Identifier'].str[:2]

In [26]:
cleanups = {"FD":"Food","DR":"Drink"}
fillItemTypeCombined = lambda x : cleanups.get(x) if cleanups.get(x) else x
data['Item_Type_Combined']=data['Item_Type_Combined'].map(fillItemTypeCombined)

In [27]:
data['Item_Type_Combined'].unique()

array(['Food', 'Drink', 'NC'], dtype=object)

In [28]:
data['Outlet_Years'] = 2013 -data['Outlet_Establishment_Year']

In [29]:
data.loc[:5,['Outlet_Establishment_Year','Outlet_Years']]

,Outlet_Establishment_Year,Outlet_Years
0,1999,14
1,2009,4
2,1999,14
3,1998,15
4,1987,26
5,2009,4


In [30]:
# change categorical variables to numeric
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 16 columns):
Item_Fat_Content              14204 non-null object
Item_Identifier               14204 non-null object
Item_MRP                      14204 non-null float64
Item_Outlet_Sales             8523 non-null float64
Item_Type                     14204 non-null object
Item_Visibility               14204 non-null float64
Item_Weight                   14204 non-null float64
Outlet_Establishment_Year     14204 non-null int64
Outlet_Identifier             14204 non-null object
Outlet_Location_Type          14204 non-null object
Outlet_Size                   14204 non-null object
Outlet_Type                   14204 non-null object
source                        14204 non-null object
Item_Visibility_Mean_Ratio    14204 non-null float64
Item_Type_Combined            14204 non-null object
Outlet_Years                  14204 non-null int64
dtypes: float64(5), int64(2), object(9)
memory usage: 1.7

In [31]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet']
for var in var_mod:
    print '\n%s:' %var
    print data[var].value_counts(dropna=False)


Item_Fat_Content:
Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

Outlet_Location_Type:
Tier 3    5583
Tier 2    4641
Tier 1    3980
Name: Outlet_Location_Type, dtype: int64

Outlet_Size:
Small     7996
Medium    4655
High      1553
Name: Outlet_Size, dtype: int64

Item_Type_Combined:
Food     10201
NC        2686
Drink     1317
Name: Item_Type_Combined, dtype: int64

Outlet_Type:
Supermarket Type1    9294
Grocery Store        1805
Supermarket Type3    1559
Supermarket Type2    1546
Name: Outlet_Type, dtype: int64

Outlet:
5    1559
1    1553
6    1550
9    1550
8    1550
7    1548
3    1546
2    1543
0     925
4     880
Name: Outlet, dtype: int64


In [32]:
dct = {"LF":"Low Fat","low fat":"Low Fat","reg":"Regular"}
fillItemFatContent = lambda x: dct.get(x) if dct.get(x) else x
data['Item_Fat_Content']=data['Item_Fat_Content'].map(fillItemFatContent)

In [33]:
data['Item_Fat_Content'].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 17 columns):
Item_Fat_Content              14204 non-null object
Item_Identifier               14204 non-null object
Item_MRP                      14204 non-null float64
Item_Outlet_Sales             8523 non-null float64
Item_Type                     14204 non-null object
Item_Visibility               14204 non-null float64
Item_Weight                   14204 non-null float64
Outlet_Establishment_Year     14204 non-null int64
Outlet_Identifier             14204 non-null object
Outlet_Location_Type          14204 non-null object
Outlet_Size                   14204 non-null object
Outlet_Type                   14204 non-null object
source                        14204 non-null object
Item_Visibility_Mean_Ratio    14204 non-null float64
Item_Type_Combined            14204 non-null object
Outlet_Years                  14204 non-null int64
Outlet                        14204 non-null int64
dtypes

In [35]:
categorical_variables = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet']
le = LabelEncoder()
for var in categorical_variables:
    data[var] = le.fit_transform(data[var])

In [36]:
# One Hot Coding
data = pd.get_dummies(data,columns =  ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'], prefix= ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type','Item_Type_Combined','Outlet'])

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14204 entries, 0 to 14203
Data columns (total 36 columns):
Item_Identifier               14204 non-null object
Item_MRP                      14204 non-null float64
Item_Outlet_Sales             8523 non-null float64
Item_Type                     14204 non-null object
Item_Visibility               14204 non-null float64
Item_Weight                   14204 non-null float64
Outlet_Establishment_Year     14204 non-null int64
Outlet_Identifier             14204 non-null object
source                        14204 non-null object
Item_Visibility_Mean_Ratio    14204 non-null float64
Outlet_Years                  14204 non-null int64
Item_Fat_Content_0            14204 non-null uint8
Item_Fat_Content_1            14204 non-null uint8
Outlet_Location_Type_0        14204 non-null uint8
Outlet_Location_Type_1        14204 non-null uint8
Outlet_Location_Type_2        14204 non-null uint8
Outlet_Size_0                 14204 non-null uint8
Outlet_Size

In [38]:
#X_train = data.loc[data['source']=='train',['Item_MRP','Item_Visibility','Item_Weight','Item_Visibility_Mean_Ratio','Outlet_Years','Item_Fat_Content_0','Item_Fat_Content_1','Outlet_Location_Type_0','Outlet_Location_Type_1','Outlet_Location_Type_2','Outlet_Size_0','Outlet_Size_1','Outlet_Size_2','']]
train = data[data['source']=='train']
test = data[data['source']=='test']

In [39]:
#X_train.drop(['Item_Outlet_Sales','Item_Identifier','Item_Type','Outlet_Identifier','Outlet_Establishment_Year','source'])b
train.drop('source',axis =1,inplace=True)
test.drop(['Item_Outlet_Sales','source'],axis =1,inplace=True)

C:\Users\huynh\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\huynh\AppData\Local\Enthought\Canopy\edm\envs\User\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [63]:
predictors = [x for x in train.columns if x not in ['Item_Type','Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']]
X_train = train[predictors]
y_train = train['Item_Outlet_Sales']
X_test = test[predictors]
test.reset_index(inplace=True,drop=True)

In [64]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators = 400,max_depth=6,min_samples_leaf=100,n_jobs=4)
model.fit(X_train,y_train)
predicted = model.predict(X_test)

In [65]:
result=pd.merge(test, pd.DataFrame(predicted,columns=['Item_Outlet_Sales']), left_index=True, right_index=True)
result.loc[:,['Item_Identifier','Outlet_Identifier','Item_Outlet_Sales']].to_csv('./dataset/big_mart_sales/result.csv',index=False)